#### Data Science Capstone Week 3 Assignment

#### Part1

Import Libraries and Download Data

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
wpage='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
rwpage= requests.get(wpage).text
soup = BeautifulSoup(rwpage,'xml')

Find and clean the table, get rid of unnecessary rows and groupby

In [3]:
t1 = soup.find('table',{'class':'wikitable sortable'})
t1r = t1.find_all('tr')

tord = []
for row in t1r:
    tord.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(tord, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

df2= df.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
df3 = df2.reset_index()
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

Number of rows and columns

In [4]:
df3.shape
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Part 2

Get Lattitude, Longitude

In [5]:
other_path = "https://cocl.us/Geospatial_data"
df4 = pd.read_csv(other_path)
df4.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Merge Dataframes

In [6]:
x1=pd.merge(df3, df4, how='inner', on=None, left_on='PostalCode', right_on='Postal Code',
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)
x1.drop(columns=['Postal Code'], inplace=True)
x1.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


#### Part 3

Import Libraries

In [9]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: / ^C
failed

CondaError: KeyboardInterrupt



ModuleNotFoundError: No module named 'folium'

#### Create a map of Toronto with neighborhoods superimposed on top.

In [10]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

neighborhoods=x1

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


folium-0.5.0         | 45 KB     | ##################################### | 1

address = 'Adelaide'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Adelaide are {}, {}.'.format(latitude, longitude))

map_t = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_t

In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
manhattan_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [16]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [17]:
CLIENT_ID = 'NIN4QMD5VVF5TXXKVWAIUC03UM05HPVEWEPFN5T2Z5PPWL3W' # your Foursquare ID
CLIENT_SECRET = '0ZX23QLLQ210ZTZNSUZ5FRTRLFF4ZRXED5FVTUUTZ5UOQV2Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NIN4QMD5VVF5TXXKVWAIUC03UM05HPVEWEPFN5T2Z5PPWL3W
CLIENT_SECRET:0ZX23QLLQ210ZTZNSUZ5FRTRLFF4ZRXED5FVTUUTZ5UOQV2Q


In [19]:
manhattan_data.loc[0, 'Neighbourhood']

'Malvern, Rouge'

In [21]:

neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


In [22]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=NIN4QMD5VVF5TXXKVWAIUC03UM05HPVEWEPFN5T2Z5PPWL3W&client_secret=0ZX23QLLQ210ZTZNSUZ5FRTRLFF4ZRXED5FVTUUTZ5UOQV2Q&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [26]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=NIN4QMD5VVF5TXXKVWAIUC03UM05HPVEWEPFN5T2Z5PPWL3W&client_secret=0ZX23QLLQ210ZTZNSUZ5FRTRLFF4ZRXED5FVTUUTZ5UOQV2Q&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [28]:
results = requests.get(url).json()


In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [38]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighbourhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [39]:
print(manhattan_venues.shape)
manhattan_venues.head()

(95, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [40]:
manhattan_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff, Cliffside West",5,5,5,5,5,5
Cedarbrae,8,8,8,8,8,8
"Clarks Corners, Tam O'Shanter, Sullivan",14,14,14,14,14,14
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile, Clairlea, Oakridge",10,10,10,10,10,10
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",8,8,8,8,8,8


In [41]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 55 uniques categories.


In [42]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighbourhood'] = manhattan_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [43]:
manhattan_onehot.shape

(95, 56)

In [45]:
manhattan_grouped = manhattan_onehot.groupby('Neighbourhood').mean().reset_index()
manhattan_grouped

,Neighbourhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Farm,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Moving Target,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Print Shop,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.000,0.000,0.000000,0.0,0.250000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.25,0.25,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.250000,0.00,0.0,0.000000,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.2,0.000,0.000000,0.000000,0.2,0.0,0.000000,0.000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.2,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.200000,0.00,0.0,0.000000,0.0
2,Cedarbrae,0.000000,0.125,0.125,0.125000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.125,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.125000,0.125000,0.0,0.000000,0.125,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.125000,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000,0.000,0.071429,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.071429,0.071429,0.0,0.0,0.071429,0.000,0.000000,0.000000,0.0,0.071429,0.071429,0.071429,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.071429,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.071429,0.0,0.0,0.071429,0.142857,0.0,0.0,0.000,0.000000,0.071429,0.000000,0.00,0.0,0.071429,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.333333,0.000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.333333,0.00,0.0,0.000000,0.0
5,"Dorset Park, Wexford Heights, Scarborough Town...",0.000000,0.000,0.000,0.000000,0.0,0.000000,0.000000,0.0,0.000,0.0,0.000,0.200000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.40,0.000,0.000000,0.00,0.00,0.00,0.000,0.0,0.000,0.00,0.000000,0.000,0.000000,0.0,0.2,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.2
6,"Golden Mile, Clairlea, Oakridge",0.000000,0.000,0.200,0.000000,0.0,0.000000,0.000000,0.2,0.100,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.1,0.00,0.100,0.000000,0.00,0.00,0.00,0.000,0.1,0.000,0.00,0.000000,0.000,0.000000,0.1,0.0,0.000000,0.000000,0.0,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.1,0.000000,0.0
7,"Guildwood, Morningside, West Hill",0.000000,0.000,0.000,0.125000,0.0,0.125000,0.000000,0.0,0.000,0.0,0.000,0.000000,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000,0.0,0.00,0.125,0.000000,0.00,0.00,0.00,0.125,0.0,0.125,0.00,0.000000,0.125,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.125,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.0
8,"Kennedy Park, Ionvie

In [46]:
manhattan_grouped.shape

(16, 56)

In [48]:
num_top_venues = 5

for hood in manhattan_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1             Breakfast Spot  0.25
2               Skating Rink  0.25
3                     Lounge  0.25
4        American Restaurant  0.00


----Birch Cliff, Cliffside West----
                   venue  freq
0           Skating Rink   0.2
1  General Entertainment   0.2
2                   Café   0.2
3                   Farm   0.2
4        College Stadium   0.2


----Cedarbrae----
                 venue  freq
0               Bakery  0.12
1                 Bank  0.12
2      Thai Restaurant  0.12
3  Fried Chicken Joint  0.12
4   Athletics & Sports  0.12


----Clarks Corners, Tam O'Shanter, Sullivan----
                 venue  freq
0          Pizza Place  0.14
1    Convenience Store  0.07
2   Chinese Restaurant  0.07
3          Gas Station  0.07
4  Fried Chicken Joint  0.07


----Cliffside, Cliffcrest, Scarborough Village West----
                 venue  freq
0  American Restaurant  0.33
1         Skat

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = manhattan_grouped['Neighbourhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


In [106]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
1,3,"Birch Cliff, Cliffside West",College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
3,3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
4,1,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


Clustering

In [142]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 3, 3, 1, 3, 3, 3, 3, 0], dtype=int32)

In [144]:
# add clustering labels
# neighborhoods_venues_sorted.drop(columns=['Cluster Labels'], inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

manhattan_merged.dropna(inplace=True)
manhattan_merged['Cluster Labels'] = manhattan_merged['Cluster Labels'].apply(np.int64)
#manhattan_merged.astype({'Cluster Labels': 'int64'}).dtypes
manhattan_merged.head(100) # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3,Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
3,M1G,Scarborough,Woburn,43.770992,-79.216917,3,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,4,Playground,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,3,Discount Store,Coffee Shop,Bus Station,Department Store,Chinese Restaurant,Hobby Shop,Convenience Store,Construction & Landscaping,Grocery Store,General Entertainment
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,3,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Intersection,Park,Bus Station,Electronics Store,Department Store
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,1,American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,3,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant


In [145]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighbourhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [146]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,0,Fast Food Restaurant,Print Shop,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Farm,Electronics Store


In [147]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,1,American Restaurant,Skating Rink,Motel,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
12,Scarborough,1,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


In [148]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


In [149]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,3,Moving Target,Bank,Breakfast Spot,Rental Car Location,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Construction & Landscaping,Gas Station
3,Scarborough,3,Coffee Shop,Indian Restaurant,Korean Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
4,Scarborough,3,Thai Restaurant,Athletics & Sports,Bakery,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store
6,Scarborough,3,Discount Store,Coffee Shop,Bus Station,Department Store,Chinese Restaurant,Hobby Shop,Convenience Store,Construction & Landscaping,Grocery Store,General Entertainment
7,Scarborough,3,Bakery,Bus Line,Ice Cream Shop,Soccer Field,Metro Station,Intersection,Park,Bus Station,Electronics Store,Department Store
9,Scarborough,3,College Stadium,General Entertainment,Skating Rink,Farm,Café,Vietnamese Restaurant,Grocery Store,Gas Station,Fried Chicken Joint,Fast Food Restaurant
10,Scarborough,3,Indian Restaurant,Vietnamese Restaurant,Pet Store,Chinese Restaurant,College Stadium,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
11,Scarborough,3,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Construction & Landscaping,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
13,Scarborough,3,Pizza Place,Gas Station,Italian Restaurant,Chinese Restaurant,Convenience Store,Noodle House,Pharmacy,Coffee Shop,Fast Food Restaurant,Shopping Mall
15,Scarborough,3,Fast Food Restaurant,Chinese Restaurant,Bubble Tea Shop,Coffee Shop,Discount Store,Electronics Store,Pharmacy,Pizza Place,Grocery Store,Breakfast Spot


In [150]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4,Playground,Construction & Landscaping,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm
14,Scarborough,4,Playground,Park,Vietnamese Restaurant,Coffee Shop,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store


#### Neighbourhoods in Cluster #3 show the most common features in Scarborough!